In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from itertools import combinations
pd.set_option('display.max_columns', 100)

In [2]:
data = pd.read_csv('senate_returns.csv')
# data = pd.read_csv('house_returns.csv')

In [3]:
committees = list(data.columns[data.columns.get_loc('Budget'):])
committees

['Budget',
 'Environment_and_Public_Works',
 'Homeland_Security_and_Governmental_Affairs',
 'Judiciary',
 'Rules_and_Administration',
 'Agriculture_Nutrition_and_Forestry',
 'Commerce_Science_and_Transportation',
 'Armed_Services',
 'Energy_and_Natural_Resources',
 'Finance',
 'Foreign_Relations',
 'Small_Business_and_Entrepreneurship',
 'Health_Education_Labor_and_Pensions',
 'Banking_Housing_and_Urban_Affairs',
 'Veterans_Affairs',
 'Appropriations']

In [4]:
data.columns

Index(['transaction_date', 'owner', 'asset_description', 'type_detailed',
       'amount', 'senator', 'ticker', 'share_class', 'type', 'return_1d',
       'return_5d', 'return_1m', 'return_index_1d', 'return_index_5d',
       'return_index_1m', 'return_abnormal_1d', 'return_abnormal_5d',
       'return_abnormal_1m', 'Senator', 'Party', 'Born', 'Occupation(s)',
       'Assumed office', 'Age', 'In_Office', 'Budget',
       'Environment_and_Public_Works',
       'Homeland_Security_and_Governmental_Affairs', 'Judiciary',
       'Rules_and_Administration', 'Agriculture_Nutrition_and_Forestry',
       'Commerce_Science_and_Transportation', 'Armed_Services',
       'Energy_and_Natural_Resources', 'Finance', 'Foreign_Relations',
       'Small_Business_and_Entrepreneurship',
       'Health_Education_Labor_and_Pensions',
       'Banking_Housing_and_Urban_Affairs', 'Veterans_Affairs',
       'Appropriations'],
      dtype='object')

In [5]:
returns = ['return_1d', 
           'return_5d',
           'return_1m', 
           'return_abnormal_1d', 
           'return_abnormal_5d', 
           'return_abnormal_1m']

# Senate columns
indep_vars = ['amount', 'type', 'Party', 'Age', 'In_Office']
# House columns
# indep_vars = ['amount', 'type', 'party', 'age', 'years_in_office']

combos = []
for i in range(len(indep_vars) + 1):
    combos.extend(list(map(lambda t: list(t), combinations(indep_vars, i))))
models = []
for ret in returns:
    models.extend(list(map(lambda c: [ret] + c, combos)))
models.extend(list(map(lambda c: c + committees, models)))

len(models)

384

In [6]:
# smf.ols('return_1d ~ Age + In_Office', data = data[['return_1d', 'amount', 'Party']].dropna()).fit().rsquared_adj

def make_model(variables):
    model_data = data[variables].dropna()
    if len(variables) == 1:
        formula = variables[0] + ' ~ 1'
    else:
        formula = variables[0] + ' ~ ' + ' + '.join(variables[1:])
    print(formula)
    model = smf.ols(formula, data = model_data).fit()
    print(model.summary())
    return formula, model.rsquared_adj

In [7]:
r2 = {}
for model in models:
    m = make_model(model)
    r2[m[0]] = m[1]

return_1d ~ 1
                            OLS Regression Results                            
Dep. Variable:              return_1d   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Fri, 18 Jun 2021   Prob (F-statistic):                nan
Time:                        23:15:58   Log-Likelihood:                 1823.5
No. Observations:                1171   AIC:                            -3645.
Df Residuals:                    1170   BIC:                            -3640.
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0022      0.001     -

                            OLS Regression Results                            
Dep. Variable:              return_1d   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.941
Date:                Fri, 18 Jun 2021   Prob (F-statistic):            0.00467
Time:                        23:15:58   Log-Likelihood:                 1833.8
No. Observations:                1171   AIC:                            -3652.
Df Residuals:                    1163   BIC:                            -3611.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

                            OLS Regression Results                            
Dep. Variable:              return_1d   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.169
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           0.000862
Time:                        23:15:58   Log-Likelihood:                 1837.7
No. Observations:                1171   AIC:                            -3655.
Df Residuals:                    1161   BIC:                            -3605.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

return_1d ~ type + Party + In_Office
                            OLS Regression Results                            
Dep. Variable:              return_1d   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.801
Date:                Fri, 18 Jun 2021   Prob (F-statistic):             0.0248
Time:                        23:15:59   Log-Likelihood:                 1829.1
No. Observations:                1171   AIC:                            -3648.
Df Residuals:                    1166   BIC:                            -3623.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:              return_1d   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     3.777
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           5.03e-05
Time:                        23:15:59   Log-Likelihood:                 1842.3
No. Observations:                1171   AIC:                            -3663.
Df Residuals:                    1160   BIC:                            -3607.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Inte

...

                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1m   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     3.365
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           7.80e-06
Time:                        23:16:13   Log-Likelihood:                 247.72
No. Observations:                1141   AIC:                            -461.4
Df Residuals:                    1124   BIC:                            -375.8
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1m   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     2.704
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           3.74e-05
Time:                        23:16:13   Log-Likelihood:                 250.61
No. Observations:                1141   AIC:                            -455.2
Df Residuals:                    1118   BIC:                            -339.3
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1m   R-squared:                       0.046
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     2.562
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           0.000138
Time:                        23:16:13   Log-Likelihood:                 247.83
No. Observations:                1141   AIC:                            -451.7
Df Residuals:                    1119   BIC:                            -340.8
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

return_abnormal_1m ~ Party + Age + In_Office + Budget + Environment_and_Public_Works + Homeland_Security_and_Governmental_Affairs + Judiciary + Rules_and_Administration + Agriculture_Nutrition_and_Forestry + Commerce_Science_and_Transportation + Armed_Services + Energy_and_Natural_Resources + Finance + Foreign_Relations + Small_Business_and_Entrepreneurship + Health_Education_Labor_and_Pensions + Banking_Housing_and_Urban_Affairs + Veterans_Affairs + Appropriations
                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1m   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     3.188
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           3.54e-05
Time:                        23:16:13   Log-Likelihood:                 244.78
No. Observations:                1141   AIC:            

                            OLS Regression Results                            
Dep. Variable:     return_abnormal_1m   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     2.712
Date:                Fri, 18 Jun 2021   Prob (F-statistic):           3.54e-05
Time:                        23:16:14   Log-Likelihood:                 250.70
No. Observations:                1141   AIC:                            -455.4
Df Residuals:                    1118   BIC:                            -339.5
Df Model:                          22                                         
Covariance Type:            nonrobust                                         
                                                 coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------

In [8]:
max(r2.values())

0.03266077957423308